# Vision Edge-AI와 3D Depth Camera

3D Depth Camera는 공간의 깊이 정보를 캡처하여 3차원 데이터로 변환하는 장치입니다. 이러한 카메라는 다양한 응용 분야에서 사용되며, 컴퓨터 비전, 로봇 공학, 증강현실(AR), 가상현실(VR), 의료 이미지 처리 등에서 중요합니다. 본 강의에서는 3D Depth Camera의 원리, 기술, 응용 사례, 그리고 DeepStream을 활용할 수 있는 방법에 대해 설명합니다.

## 03-2. 3D Depth Camera

### 3D Depth Camera의 정의
3D Depth Camera는 객체나 장면의 깊이 정보를 캡처할 수 있는 장치로, 카메라가 각 픽셀의 거리 값을 캡처하여 3차원 공간에서 물체를 인식하거나 측정할 수 있도록 합니다.

### 3D Depth Camera의 원리
1. **스테레오 비전 (Stereo Vision)**:
    * 두 개의 카메라로 물체를 촬영한 이미지를 비교하여 깊이를 계산합니다.
    * 사람의 시각 시스템과 유사한 방식입니다.

2. **구조광 (Structured Light)**:
    * 특정 패턴의 빛을 물체에 투사하고, 왜곡된 패턴을 분석하여 깊이를 측정합니다.
    * 대표적인 예: Microsoft Kinect

3. **비행시간 (Time of Flight, ToF)**:
    * 빛이 물체에 반사되어 돌아오는 시간을 측정하여 깊이를 계산합니다.
    * 정확하고 빠른 깊이 측정이 가능합니다.

4. **LiDAR (Light Detection and Ranging)**:
    * 레이저를 사용해 주변 환경을 스캔하여 깊이를 측정합니다.
    * 자율주행차에 주로 사용됩니다.

### 응용 분야
1. **산업 및 제조**:
    * 로봇 비전: 로봇의 정확한 동작과 위치 결정을 지원.
    * 품질 검사: 제품의 결함 및 크기 측정.
2. **의료**:
    * 3D 스캔: 인체를 스캔하여 맞춤형 의료 장비 제작.
    * 수술 보조: 정확한 수술 계획을 위한 3D 모델 생성.
3. **엔터테인먼트**:
    * 증강현실/가상현실(AR/VR): 몰입형 환경 구축.
    * 게임: 사용자 동작을 추적하여 인터랙션 제공.
4. **도매(Retail)**:
    * 고객 행동 분석: 매장 내 고객의 동선을 추적하고 행동을 분석.
    * 재고 관리: 제품 크기 및 위치를 자동으로 스캔하여 재고를 효율적으로 관리.
    * 무인 매장: 3D 데이터를 활용해 고객과 제품의 상호작용을 자동화.

## 03-3. OAK-D 모듈
- **OAK-D 모듈 소개**: OAK-D(OpenCV AI Kit Depth)는 AI 및 컴퓨터 비전 기능을 갖춘 카메라 모듈로, 실시간으로 깊이 정보를 제공하며 다양한 응용 분야에 적합합니다.

- **주요 특징**:
  1. **내장형 AI**: Myriad X VPU를 탑재하여 온보드 AI 처리 가능.
  2. **스테레오 비전**: 두 개의 모노 카메라로 깊이 정보 계산.
  3. **컬러 카메라**: 고해상도 컬러 카메라로 세부 정보를 캡처.
  4. **플러그 앤 플레이**: USB를 통해 쉽게 연결 및 사용 가능.
  5. **다양한 SDK 지원**: DepthAI 및 OpenCV와 같은 SDK 지원.

- **응용 사례**:
  - 로봇 내비게이션
  - 객체 추적 및 감지
  - 증강 현실 및 가상 현실 애플리케이션

---

# 간단한 예제 실습

## Jetson Orin Nano에서 OAK-D PRO setting (Connection)

### 준비물
- Jetson Orin Nano
- OAK-D PRO Camera
- USB-C 케이블

### 필수 라이브러리 설치
라이브러리 설치
```bash
# 설치되어있는 OpenCV 삭제
$ pip uninstall opencv-python

# 저장소 추가
$ sudo add-apt-repository universe
$ sudo apt update

# 필수 라이브러리 설치
$ sudo apt install -y build-essential cmake git pkg-config libgtk-3-dev libavcodec-dev libavformat-dev libswscale-dev libv4l-dev libxvidcore-dev libx264-dev libjpeg-dev libpng-dev libtiff-dev gfortran openexr libatlas-base-dev python3-dev python3-numpy libtbb2 libtbb-dev libdc1394-dev

# OpenCV 설치
pip install opencv-python
```

OAK-D 관련 드라이버 설치
```bash
$ wget -qO- https://docs.luxonis.com/install_dependencies.sh | bash
$ pip install depthai --upgrade
```
* 드라이버 설치 이후 반드시 OAK-D 모듈 연결 USB를 뺐다가 다시 꽂기

In [1]:
import cv2
import depthai as dai

# Create pipeline
pipeline = dai.Pipeline()

# Create ColorCamera node
cam = pipeline.create(dai.node.ColorCamera)
cam.setBoardSocket(dai.CameraBoardSocket.CAM_A)  # CAM_A로 수정
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
cam.setVideoSize(1920, 1080)

# Create XLinkOut node
xout = pipeline.create(dai.node.XLinkOut)
xout.setStreamName("video")

# Connect camera to output
cam.video.link(xout.input)

# Start the pipeline
with dai.Device(pipeline) as device:
    print("Pipeline started")
    video = device.getOutputQueue(name="video", maxSize=4, blocking=False)
    
    while True:
        # Get frame
        frame = video.get().getCvFrame()
        
        # Show frame
        cv2.imshow("OAK-D Video Feed", frame)
        
        # Exit on 'q' key
        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()

Pipeline started


## OAK-D 모듈을 활용한 ObjectDetection

### 준비물
* Linux Ubuntu 22.04 
* OAK-D 카메라 모듈

### 실행 준비
라이브러리 설치
```bash
sudo apt update

# C++ 표준 라이브러리
sudo apt install build-essential g++ libstdc++-12-dev

# nlohmann-json 라이브러리 설치
sudo apt install nlohmann-json3-dev
```